In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# === Config ===
INPUT_FILE = "EQUITY_L.csv"
OUTPUT_FILE = "classified_stocks.csv"
BASE_URL = "https://www.screener.in/company/"
DELAY = 60/40


# Load CSV and initialize Sector column
df = pd.read_csv(INPUT_FILE)
df['Symbol'] = df['SYMBOL'].str.upper().str.strip()
df['Sector'] = ""

# Extract Sector from Screener Breadcrumb
def get_sector_breadcrumb(symbol):
    try:
        url = f"{BASE_URL}{symbol}/"
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # Get ALL <a> tags from breadcrumb area
        breadcrumb_div = soup.find("div", class_="flex flex-space-between")
        if not breadcrumb_div:
            return "N/A"

        a_tags = breadcrumb_div.find_all("a")
        sectors = [a.text.strip() for a in a_tags if a.get("href", "").startswith("/market/")]

        if sectors:
            return " > ".join(sectors)
        return "N/A"
    except Exception as e:
        print(f"[{symbol}] Error: {e}")
        return "N/A"

# Loop through all stocks and extract sector
for idx, row in df.iterrows():
    symbol = row['Symbol']
    sector = get_sector_breadcrumb(symbol)
    df.at[idx, 'Sector'] = sector
    print(f"{symbol}: {sector}")
    time.sleep(DELAY)

# Save result
df.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Done! Sectors saved to: {OUTPUT_FILE}")


20MICRONS: Commodities > Metals & Mining > Minerals & Mining > Industrial Minerals
21STCENMGM: Financial Services > Financial Services > Capital Markets > Other Capital Market related Services
360ONE: Financial Services > Financial Services > Capital Markets > Stockbroking & Allied
3IINFOLTD: Information Technology > Information Technology > IT - Software > Computers - Software & Consulting
3MINDIA: Diversified > Diversified > Diversified > Diversified
3PLAND: Financial Services > Financial Services > Finance > Investment Company
5PAISA: Financial Services > Financial Services > Capital Markets > Stockbroking & Allied
63MOONS: Information Technology > Information Technology > IT - Software > Computers - Software & Consulting
A2ZINFRA: Industrials > Construction > Construction > Civil Construction
AAATECH: Services > Services > Commercial Services & Supplies > Consulting Services
AADHARHFC: Financial Services > Financial Services > Finance > Housing Finance Company
AAKASH: Energy > Oil,

In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("classified_stocks.csv")

# Count 'N/A' entries in Sector column
na_count = df['Sector'].isna().sum()
print(f"🟡 Number of 'N/A' entries in Sector column: {na_count}")


🟡 Number of 'N/A' entries in Sector column: 19


In [ ]:
missing_sectors = df[df['Sector'].isna()]
print(missing_sectors)

          SYMBOL                           NAME OF COMPANY  SERIES  \
13         AARON                  Aaron Industries Limited      EQ   
70        AHLADA                  Ahlada Engineers Limited      EQ   
415        CROWN                     Crown Lifters Limited      EQ   
502      DPSCLTD                              DPSC Limited      EQ   
676        GODHA         Godha Cabcon & Insulation Limited      EQ   
754   HECPROJECT                HEC Infra Projects Limited      BE   
810      ICEMAKE            Ice Make Refrigeration Limited      EQ   
949          JMA    Jullundur Motor Agency (Delhi) Limited      EQ   
1069     LATTEYS                Latteys Industries Limited      BE   
1075       LEXUS             Lexus Granito (India) Limited      EQ   
1113    MACPOWER             Macpower CNC Machines Limited      BE   
1125  MAHLIFE-RE  Mahindra Lifespace Developers Limited-RE      BE   
1156      MARINE        Marine Electricals (India) Limited      EQ   
1314   NOIDATOLL    

In [ ]:
!pip install ace_tools

In [ ]:
import pandas as pd


# Load your file
df = pd.read_csv("classified_stocks.csv")

# Split the 'Sector' column by ' > ' into multiple columns
split_sectors = df['Sector'].str.split(" > ", expand=True)

# Rename the new columns dynamically as 'Level 1', 'Level 2', ...
split_sectors.columns = [f'Level {i+1}' for i in range(split_sectors.shape[1])]

# Concatenate the split columns with the original dataframe
df_combined = pd.concat([df, split_sectors], axis=1)

# Save to CSV if needed
df_combined.to_csv("sector_hierarchy.csv", index=False)




In [ ]:
df = pd.read_csv("sector_hierarchy.csv")


In [ ]:
# unique Level 1 sectors
df['Level 3'].unique()


array(['Minerals & Mining', 'Capital Markets', 'IT - Software',
       'Diversified', 'Finance', 'Construction',
       'Commercial Services & Supplies', 'Oil',
       'Pharmaceuticals & Biotechnology', nan, 'Electrical Equipment',
       'Chemicals & Petrochemicals', 'Textiles & Apparels', 'Beverages',
       'Retailing', 'Paper, Forest & Jute Products',
       'Cement & Cement Products', 'Transport Services',
       'Agricultural, Commercial & Construction Vehicles', 'Power',
       'Metals & Minerals Trading', 'Transport Infrastructure',
       'Food Products', 'Consumer Durables', 'Industrial Products',
       'IT - Services', 'Leisure Services', 'Gas',
       'Industrial Manufacturing', 'Healthcare Services', 'Realty',
       'Agricultural Food & other Products',
       'Fertilizers & Agrochemicals', 'Financial Technology (Fintech)',
       'Telecom -  Equipment & Accessories', 'Auto Components',
       'Household Products', 'Ferrous Metals', 'Consumable Fuels',
       'Aerospace 

In [ ]:
# give me the list of stocks under 'commodities' in level 1
df[df['Level 1'] == 'Commodities']


,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE,Symbol,Sector,Level 1,Level 2,Level 3,Level 4
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5,20MICRONS,Commodities > Metals & Mining > Minerals & Min...,Commodities,Metals & Mining,Minerals & Mining,Industrial Minerals
16,AARTIIND,Aarti Industries Limited,EQ,08-FEB-1995,5,1,INE769A01020,5,AARTIIND,Commodities > Chemicals > Chemicals & Petroche...,Commodities,Chemicals,Chemicals & Petrochemicals,Specialty Chemicals
18,AARTISURF,Aarti Surfactants Limited,BE,14-JUL-2020,10,1,INE09EO01013,10,AARTISURF,Commodities > Chemicals > Chemicals & Petroche...,Commodities,Chemicals,Chemicals & Petrochemicals,Specialty Chemicals
30,ABREL,Aditya Birla Real Estate Limited,EQ,27-JUN-2003,10,1,INE055A01016,10,ABREL,"Commodities > Forest Materials > Paper, Forest...",Commodities,Forest Materials,"Paper, Forest & Jute Products",Paper & Paper Products
32,ACC,ACC Limited,EQ,20-NOV-1996,10,1,INE012A01025,10,ACC,Commodities > Construction Materials > Cement ...,Commodities,Construction Materials,Cement & Cement Products,Cement & Cement Products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,VISHNU,Vishnu Chemicals Limited,EQ,05-MAR-2015,2,1,INE270I01022,2,VISHNU,Commodities > Chemicals > Chemicals & Petroche...,Commodities,Chemicals,Chemicals & Petrochemicals,Specialty Chemicals
2061,VRAJ,Vraj Iron and Steel Limited,EQ,03-JUL-2024,10,1,INE0S2V01010,10,VRAJ,Commodities > Metals & Mining > Ferrous Metals...,Commodities,Metals & Mining,Ferrous Metals,Sponge Iron
2097,WSTCSTPAPR,West Coast Paper Mills Limited,EQ,17-JAN-2001,2,1,INE976A01021,2,WSTCSTPAPR,"Commodities > Forest Materials > Paper, Forest...",Commodities,Forest Materials,"Paper, Forest & Jute Products",Paper & Paper Products
2103,YASHO,Yasho Industries Limited,EQ,21-AUG-2023,10,1,INE616Z01012,10,YASHO,Commodities > Chemicals > Chemicals & Petroche...,Commodities,Chemicals,Chemicals & Petrochemicals,Specialty Chemicals
